# Programmazione imperativa in OCaml (cenni)

In questo capitolo saranno illustrati brevemente alcuni aspetti di programmazione imperativa presenti nel linguaggio OCaml.

### Riferimenti

Questo capitolo si basa su materiale ed esempi di codice tratti da:

- <A href="https://dev.realworldocaml.org/">Real World OCaml</A> (Versione 2), di Yaron Minsky, Anil Madhavapeddy e Jason Hickey

La licenza per l'utilizzo del testo e degli esempi di codice è disponibile nel sito indicato.

Inoltre, alcuni aspetti del linguaggio possono essere approfonditi consultando il manuale ufficiale di OCaml, disponibile a questo indirizzo:

- <A href="https://ocaml.org/manual/index.html">https://ocaml.org/manual/index.html</A>

## Introduzione

Fino ad ora abbiamo scritto codice secondo il paradigma funzionale: 

Abbiamo principalmente definito *funzioni pure*

- Calcolano il risultato sulla base dei propri parametri e delle variabili disponibili nella chiusura

e variabili/strutture dati *immutabili*

- Non è possibile modificarle assegnando nuovi valori

Nella *programmazione imperativa* invece:

* le variabili sono *mutabili* (si possono assegnare a piacimento)
* le funzioni causano *side effect* (possono modificare l'ambiente globale)

Esempi di side effects:

* lettura/assegnamento di una varibile globale (o non locale)
* stampa di un messaggio di output / lettura di un input
* sollevamento di una eccezione 

Tutto questo rende le funzioni *impure*

* il loro effetto non è più solo calcolare un risultato
* il risultato può essere influenzato da fattori esterni (variabili globali/input/...)


OCaml è concepito principalmente per la programmazione funzionale...

* .. ma include anche tutti i *costrutti imperativi* principali

Ne accenneremo brevemente alcuni:

* *array* e *record mutabili*
* *riferimenti*
* *cicli*
* *eccezioni*

Inoltre, OCaml prevede la possibilità di definire *classi e oggetti* in pieno stile object-oriented

* questo per il momento non lo vediamo

## Array

Gli array in OCaml:

- contengono tutti elementi dello stesso tipo
- hanno dimensione fissata al momento della creazione  (come in C, Java, ...)
- possono essere acceduti tramite indici numerici (non sono associativi)

Definizione di un nuovo array:
   
- letterale (elencandone gli elementi tra `[|` e `|]`

In [1]:
let a = [|3;5;2|] ;;

val a : int array = [|3; 5; 2|]


- tramite `Array.make` (con parametri la dimensione e il valore iniziale)

In [2]:
Array.make 10 0;;

- : int array = [|0; 0; 0; 0; 0; 0; 0; 0; 0; 0|]


La lunghezza di un array può essere ottenuta tramite `Array.length`

In [3]:
let n = Array.length a;;

val n : int = 3


Accedere agli elementi di un array (con indici in `0,...,(n-1)`):

- in lettura, con la sintassi `.(i)`

In [4]:
let e = a.(1) ;;

val e : int = 5


- in scrittura, con la sintassi `<-`

In [5]:
a.(1) <- 6 ;;
a;;

- : unit = ()


- : int array = [|3; 6; 2|]


### Un commento su `<-`

Il costrutto `<-` è un *COMANDO di ASSEGNAMENTO*

- è la prima volta che *modifichiamo* qualcosa!

Non è l'unico costrutto di assegnamento

- ne vedremo altri con sintassi diversa

Il *tipo dei comandi* in OCaml è `unit`

- in realtà non sono comandi, ma *espressioni con side effect*
- simile a `void` in altri linguaggi di programmazione
- descrive le cose che non possono essere valutate a valori
- l'unico valore del tipo `unit` è `()`


## Record mutabili

E' possibile rendere mutabili uno o più campi tramite il modificatore `mutable`

In [6]:
type persona = 
{
    nome: string;
    cognome: string;
    mutable eta: int;
}

type persona = { nome : string; cognome : string; mutable eta : int; }


In [7]:
let mario = {nome="mario"; cognome="rossi"; eta=30 } ;;

val mario : persona = {nome = "mario"; cognome = "rossi"; eta = 30}


Anche in questo caso per l'assegnamento si usa `<-`

In [8]:
mario.eta <- 31 ;;
mario ;;

- : unit = ()


- : persona = {nome = "mario"; cognome = "rossi"; eta = 31}


In [9]:
let invecchia p =
    p.eta <- p.eta + 1;;
    
invecchia mario;;
mario ;;

val invecchia : persona -> unit = <fun>


- : unit = ()


- : persona = {nome = "mario"; cognome = "rossi"; eta = 32}


## Riferimenti (refs)

Oltre ad array e record mutabili, è possibile definire anche singole variabili mutabili usando il tipo `ref`

Ad esempio:

In [10]:
let x = ref 12 ;;

val x : int ref = {contents = 12}


Si definisce la variabile `x` che contiene un *riferimento*

* il riferimento punta a un record con un solo campo mutabile `contents` inizializato con il valore passato

E' possibile accedere alla variabile `ref` come record

In [11]:
x.contents ;;
x.contents <- 13;;

- : int = 12


- : unit = ()


Ma in realtà c'è una *sintassi specifica*, più semplice:

In [12]:
!x ;;            (* corrisponde a x.contents *)
x := 14 ;;       (* corrisponde a x.contents <- 14 *)

- : int = 13


- : unit = ()


L'operazione `!` è detta *dereferenziazione*

Con `ref` si creano effettivamente dei riferimenti

* dei puntatori ad aree di memoria

Copiando il riferimento in una nuova variabile, entrambe le variabili punteranno alla stessa area di memoria

In [13]:
let x = ref 0;;
let y = x;;
y:=100;;
!x;;

val x : int ref = {contents = 0}


val y : int ref = {contents = 0}


- : unit = ()


- : int = 100


Modificando `y` anche `x` risulta modificata

## Sequenze di comandi e cicli

Una sequenza di comandi/espressioni può essere definita tramite il separatore `;`

In [14]:
let x = ref 0;;
let y = x;;
x := !x+1 ; y := !y+1 ; (!x,!y) ;; (* sequenza di espressioni *)

val x : int ref = {contents = 0}


val y : int ref = {contents = 0}


- : int * int = (2, 2)


Le espressioni vengono valutate una dopo l'altra e l'ultima fornisce il risultato finale

In una sequenza di espressioni `E1 ; E2 ; .... ; En`:

- tutte le espressioni precedenti all'ultima devono (dovrebbero...) avere tipo `unit`
- le espressioni `E1 ; E2 ; .... ; E(n-1)` causano solo side effects (ad esempio, modificando le strutture dati mutabili)
- l'espressione `En` calcola il risultato vero e proprio

In [15]:
x := !x+1 ; y := !y+1 ; (!x,!y) ;;

- : int * int = (4, 4)


In [16]:
4+3 ; 5+2 ;;

File "[16]", line 1, characters 0-3:
1 | 4+3 ; 5+2 ;;
    ^^^
Warning 10: this expression should have type unit.
File "[16]", line 1, characters 0-3:
1 | 4+3 ; 5+2 ;;
    ^^^
Warning 10: this expression should have type unit.


- : int = 7


Sfruttando la funzione `print_endline` che stampa una stringa sulla console, il sequenziamento `;` può essere utile per stampare *messaggi di debug*

In [17]:
let abs x =
    if x>0 then (
        print_endline "ramo THEN" ;
        x
    )
    else (
        print_endline "ramo ELSE" ;
        -x
    );;

abs (-5) ;;

val abs : int -> int = <fun>


ramo ELSE


- : int = 5


In [18]:
let abs x =
    if x>0 then begin
        print_endline "ramo THEN" ;
        x
    end
    else begin
        print_endline "ramo ELSE" ;
        -x
    end;;

abs (-5) ;;

val abs : int -> int = <fun>


ramo ELSE


- : int = 5


**NOTA**: le parentesi tonde e `begin...end` sono modi alternativi per creare un "blocco"

### TODO: Spiegare brevemente

### NOTA: `;` vs `in`

Viene la tentazione di usare `;` per separare dichiarazioni di variabili dal loro uso...

In [46]:
let f n = n+1 ;; (* funzione giocattolo, giusto per l'esempio *)
let m = 10 ; f(m) ;;

val f : int -> int = <fun>


File "[46]", line 2, characters 8-10:
2 | let m = 10 ; f(m) ;;
            ^^
Warning 10: this expression should have type unit.


error: compile_error

invece bisogna usare `in`

In [20]:
let m = 10 in f(m) ;;

- : int = 11


### Cicli `for`

La sintassi del comando `for` è la seguente:

```
for <variable> = <start> to <end> do
  ...
done
```

oppure 

```
for <variable> = <start> downto <end> do
  ...
done
```

dove `<start>` e `<end>` sono valori interi entro cui far variare la `<variabile>`

Esempio giocattolo:

In [21]:
for i = 1 to 10 do
    print_endline (string_of_int i)
done ;;

1
2
3
4
5
6
7
8
9
10


- : unit = ()


Esempio: media dei valori di un array

In [22]:
let media arr =
    let s = ref 0 in
    for i = 0 to (Array.length arr)-1 do
        s := !s + arr.(i)
    done ;
    !s / Array.length arr ;;
    
media [|4;6;8|] ;;

val media : int array -> int = <fun>


- : int = 6


### Cicli `while`

La sintassi del comando `while` è la seguente:

```
while <condizione> do
  ...
done
```

dove `<condizione>` è una espressione di tipo `boolean`

In [23]:
let x=ref 10 in
while !x>0 do
    print_endline (string_of_int !x) ;
    x := !x/2
done

10
5
2
1


- : unit = ()


Esempio: restituisce la posizione del primo valore negativo in un array, se presente

In [24]:
let primo_negativo arr =
    let pos = ref 0 in
    let trovato = ref false in 
    while !pos < Array.length arr && not !trovato do
        if arr.(!pos)<0 
            then trovato := true
            else pos := !pos + 1
    done;
    if (!trovato) 
        then Some !pos
        else None ;;
    
primo_negativo [| 3; 4; -1; 6; -3; -6 |] ;;

val primo_negativo : int array -> int option = <fun>


- : int option = Some 2


### Commenti sui cicli...

OCaml *non incoraggia* l'uso dei cicli:

- la sintassi non è molto *friendly*
- non è possibile interromperli brutalmente (non esiste `break` o `return`)
- il `for` non è flessibile sull'uso degli indici (no incremento di 2 per volta)

D'altra parte *incoragga* fortemente l'uso di un approccio funzionale

- ottimizzazione delle funzioni *tail-recursive*
- funzioni `iter`, `for_all`,`exists`,`map`,`filter`,`fold_right`,`fold_left` in `List` e `Array` per "ciclare" su liste e array


## Eccezioni

Le eccezioni sono un modo usato in molti linguaggi di programmazione per gestire le situazioni anomale e di errore nei programmi

In [25]:
3 / 0 ;;

error: runtime_error

### TODO: richiamare il funzionamento delle eccezioni
il fatto che interrompono il programma, ecc...
richiamare anche come funzionavano in JS?

Esistono vari tipi di eccezioni predefinite (es. `Division_by_zero`) e si possono  definire le proprie
* specificando un costruttore e, facoltativamente, un tipo (come nei tipi variant)

In [26]:
exception Lista_vuota ;;
exception Stringa_errata of string ;;

exception Lista_vuota


exception Stringa_errata of string


Le eccezioni si sollevano con `raise`...

In [27]:
raise Lista_vuota;;

error: runtime_error

In [28]:
raise (Stringa_errata "test") ;;

error: runtime_error

... e possono essere usate, ad esempio, per interrompere una funzione in caso di errore

In [29]:
let minimo lis =
    let rec minimo_ric m lis' =
        match lis' with
        | [] -> m
        | x::lis' -> minimo_ric (if x<m then x else m) lis'
    in 
    match lis with
    | [] -> raise Lista_vuota
    | x::lis' -> minimo_ric x lis'
;;

val minimo : 'a list -> 'a = <fun>


In [30]:
minimo [4;3;5;6;2;9] ;;

- : int = 2


In [31]:
minimo [] ;;

error: runtime_error

Altro esempio: giorni della settimana

In [32]:
let giorno_num g =
    match g with
    | "lun" -> 1 | "mar" -> 2 | "mer" -> 3 | "gio" -> 4 
    | "ven" -> 5 | "sab" -> 6 | "dom" -> 7
    | _ -> raise (Stringa_errata g) ;;

val giorno_num : string -> int = <fun>


In [33]:
giorno_num "gio" ;;

- : int = 4


In [34]:
giorno_num "lunedì" ;;

error: runtime_error

Inoltre, è possibile intercettare e gestire eccezioni con il costrutto `try ... with`.

La sintassi (simile a quella del pattern matching) è la seguente:

```
try <espressione> with
| <pat1> -> <esp1>
| <pat2> -> <esp2>
...
| <patN> -> <espN>
```

e la semantica è che se `<espressione>` causa un'eccezione che fa match con il pattern `<patI>`, viene valutata l'espressione `<espI>`

* *NOTA:* non c'è controllo di esaustività dei pattern

Nel costrutto `try ... with` non avrebbe senso testare l'esaustività dei pattern, in quanto, essendo le eccezioni personalizzabili, le possibili eccezioni che possono essere sollevate sono infinite. L'unico modo per raggiungere l'esaustività sarebbe di includere sempre il pattern wildcard `_` come ultimo caso possibile.

Inoltre, spesso i gestori di eccezioni definiti con il costrutto `try ... with` si preoccupano di catturare una o un gruppo specifico di eccezioni, lasciando che eventuali altre "passino" venendo poi cattute da un controllo più esterno, oppure portando all'interruzione del programma, senza essere gestite internamente. 

Per tutti questi motivi un controllo di esaustività di questi pattern sarebbe privo di senso.

Esempio (un po' forzato, si farebbe `List.for_all (fun x -> x>=0) lis`):

In [35]:
let tutti_positivi lis =
  try 
    minimo lis >= 0
  with
  | Lista_vuota -> true ;;

val tutti_positivi : int list -> bool = <fun>


In [36]:
tutti_positivi [3;2;-5];;

- : bool = false


In [37]:
tutti_positivi [];;

- : bool = true


La funzione `tutti_positivi` verifica se gli elementi di un array sono tutti positivi andandone a calcolare il minimo con la funzione `minimo` definita in precedenza. Qualora la lista sia vuota, l'eccezione sollevata dalla funzione `minimo` viene catturata e il risultato sarà `true` (Nota: qualunque proprietà è soddisfatta da tutti gli elementi di una lista vuota...)

Altro esempio: trova il minimo (come numero) in una lista di giorni in formato testuale

In [47]:
let primo_giorno lis =
    try
        Some (minimo (List.map giorno_num lis))   (* giorno_num definita in precedenza *)
    with
    | Stringa_errata s -> print_endline ("Stringa errata: "^s) ; None
    | Lista_vuota -> print_endline "Lista vuota" ; None
    | _ -> print_endline "Errore" ; None    (* caso _ aggiunto cattura eventuali altre eccezioni... *)

val primo_giorno : string list -> int option = <fun>


In [39]:
primo_giorno ["sab"; "gio"; "ven"] ;;

- : int option = Some 4


In [40]:
primo_giorno ["sabato"; "giovedì"; "venerdì"] ;;

Stringa errata: sabato


- : int option = None


In [41]:
primo_giorno [] ;;

Lista vuota


- : int option = None


Anche in questo esempio sfruttiamo l'eccezione sollevata da `minimo`. Innanzitutto, la lista di stringhe viene data in pasto alla funzione `List.map` che, applicando la funzione `giorno_num` ad ogni elemento, produce una lista di numeri che rappresentano i vari giorni. A questo punto `minimo` calcola il numero minimo, che viene restituito come tipo `int option`, quindi preceduto dal costruttore `Some`.

Le funzioni `giorno_map` e `minimo` possono sollevare eccezioni. La prima nel caso in cui una stringa sia scritta in modo errato, e la seconda nel caso in cui la lista sia vuota. Entrambe le eccezioni vengono gestite tramite il costrutto `try ... with` visualizzando un messaggio di errore e restituendo `None`. In questo caso, sebbene non sia essenziale, è stato aggiungo anche il pattern `_` per cattuare eventuali altre eccezioni (che comunque non dovrebbero verificarsi).

Ocaml fornisce inoltre alcuni altri costrutti per sollevare eccezioni alternativi a `raise`

In [42]:
failwith "messaggio di errore" ;;

error: runtime_error

corrisponde a `raise (Failure "messaggio di errore")`, ma più breve

Inoltre, `assert <condizione>` solleva un'eccezione se `<condizione>` è falsa

In [43]:
let dividi n m = assert (m<>0) ; n/m ;;

val dividi : int -> int -> int = <fun>


In [44]:
dividi 4 2;;

- : int = 2


In [45]:
dividi 4 0;;

error: runtime_error

L'eccezione sollevata da `assert` fornisce indicazioni sulla riga di codice in cui si trova la condizione che è stata violata.